# Ca11: Advanced Model-based Rl and World Models
## Deep Reinforcement Learning - Session 11

### Course Information
- **Course**: Deep Reinforcement Learning
- **Session**: 11
- **Topic**: Advanced Model-Based RL and World Models
- **Focus**: World models, latent space planning, and modern model-based approaches

### Learning Objectives

By the end of this notebook, you will understand:

1. **World Model Foundations**:
- Variational autoencoders for state compression
- Latent dynamics modeling and prediction
- Reward modeling in compressed state space
- Uncertainty quantification in world models

2. **Recurrent State Space Models**:
- Temporal dependencies in world modeling
- Recurrent neural networks for state evolution
- Memory-augmented latent representations
- Long-term prediction and imagination

3. **Planning in Latent Space**:
- Actor-critic methods in compressed representations
- Imagination-based planning and rollout
- Model-based policy optimization
- Sample efficiency through latent planning

4. **Dreamer Architecture**:
- Complete Dreamer agent implementation
- World model learning and imagination
- Latent actor-critic training
- End-to-end model-based RL pipeline

5. **Advanced Techniques**:
- Stochastic vs deterministic dynamics
- Ensemble methods for uncertainty
- Contrastive learning for representations
- Meta-learning with world models

6. **Implementation Skills**:
- Modular world model architecture design
- Latent space policy learning
- World model training and evaluation
- Scalable model-based RL systems

### Prerequisites

Before starting this notebook, ensure you have:

- **Mathematical Background**:
- Variational inference and autoencoders
- Recurrent neural networks and LSTMs
- Latent variable models and representation learning
- Stochastic processes and uncertainty modeling

- **Programming Skills**:
- Advanced PyTorch (custom architectures, training loops)
- Neural network debugging and optimization
- GPU acceleration and memory management
- Modular code design and testing

- **Reinforcement Learning Knowledge**:
- Model-based RL fundamentals (from CA10)
- Actor-critic methods and policy gradients
- Experience replay and off-policy learning
- Continuous control and action spaces

- **Previous Course Knowledge**:
- CA1-CA9: Complete RL fundamentals and algorithms
- CA10: Model-based RL and planning methods
- Strong foundation in PyTorch and neural architectures
- Experience with complex RL implementations

### Roadmap

This notebook follows a structured progression from world modeling to complete agents:

1. **Section 1: World Models and Latent Representations** (60 min)
- Variational autoencoder fundamentals
- Latent dynamics and reward modeling
- World model training and evaluation
- Uncertainty quantification techniques

2. **Section 2: Recurrent State Space Models** (45 min)
- Temporal world modeling with RNNs
- RSSM architecture and training
- Memory-augmented representations
- Long-horizon prediction capabilities

3. **Section 3: Dreamer Agent - Planning in Latent Space** (60 min)
- Latent actor-critic architecture
- Imagination-based planning
- Dreamer training pipeline
- Performance analysis and evaluation

4. **Section 4: Running Complete Experiments** (45 min)
- Experiment configuration and setup
- Training world models end-to-end
- Evaluation protocols and metrics
- Hyperparameter tuning strategies

5. **Section 5: Key Benefits of Modular Design** (30 min)
- Code organization and reusability
- Testing and debugging strategies
- Extensibility and maintenance
- Research and development workflows

### Project Structure

This notebook uses a modular implementation organized as follows:

```
CA11/
├── world_models/             # World model components
│   ├── vae.py               # Variational Autoencoder
│   ├── dynamics.py          # Latent dynamics models
│   ├── reward_model.py      # Reward prediction models
│   ├── world_model.py       # Complete world model
│   ├── rssm.py              # Recurrent State Space Model
│   └── trainers.py          # Model training utilities
├── agents/                   # RL agents
│   ├── latent_actor.py      # Latent space actor networks
│   ├── latent_critic.py     # Latent space critic networks
│   ├── dreamer_agent.py     # Complete Dreamer agent
│   └── utils.py             # Agent utilities
├── environments/             # Custom environments
│   ├── continuous_cartpole.py # Continuous cartpole
│   ├── continuous_pendulum.py # Continuous pendulum
│   ├── sequence_environment.py # Sequence prediction tasks
│   └── wrappers.py           # Environment wrappers
├── utils/                    # General utilities
│   ├── data_collection.py   # Experience collection tools
│   ├── visualization.py     # Plotting and analysis
│   ├── evaluation.py        # Performance evaluation
│   └── helpers.py           # Helper functions
├── experiments/              # Complete experiment scripts
│   ├── world*model*experiment.py # World model training
│   ├── rssm_experiment.py   # RSSM training experiments
│   ├── dreamer_experiment.py # Full Dreamer training
│   ├── ablation_study.py    # Component analysis
│   └── hyperparameter_sweep.py # Parameter optimization
├── configs/                  # Configuration files
│   ├── world*model*config.py # World model settings
│   ├── dreamer_config.py    # Dreamer agent settings
│   ├── environment_configs.py # Environment parameters
│   └── training_configs.py  # Training hyperparameters
├── tests/                    # Unit tests
│   ├── test*world*models.py # World model tests
│   ├── test_agents.py       # Agent tests
│   ├── test_environments.py # Environment tests
│   └── test_utils.py        # Utility tests
├── requirements.txt          # Python dependencies
├── setup.py                 # Package setup
├── README.md                # Project documentation
└── CA11.ipynb              # This educational notebook
```

### Contents Overview

1. **Section 1**: World Models and Latent Representations
2. **Section 2**: Recurrent State Space Models (RSSM)
3. **Section 3**: Dreamer Agent - Planning in Latent Space
4. **Section 4**: Running Complete Experiments
5. **Section 5**: Key Benefits of Modular Design

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from models.vae import VariationalAutoencoder
from models.dynamics import LatentDynamicsModel
from models.reward_model import RewardModel
from models.world_model import WorldModel
from models.rssm import RecurrentStateSpaceModel
from models.trainers import WorldModelTrainer, RSSMTrainer

from agents.latent_actor import LatentActor
from agents.latent_critic import LatentCritic
from agents.dreamer_agent import DreamerAgent

from environments.continuous_cartpole import ContinuousCartPole
from environments.continuous_pendulum import ContinuousPendulum
from environments.sequence_environment import SequenceEnvironment

from utils.data_collection import collect_world_model_data, collect_sequence_data
from utils.visualization import plot_world_model_training, plot_rssm_training

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Advanced Model-Based RL Environment Setup")
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)
colors = sns.color_palette("husl", 8)
sns.set_palette(colors)

print("✅ Modular environment setup complete!")
print("🌟 Ready for advanced model-based reinforcement learning!")


# Section 1: World Models and Latent Representations

## 1.1 Understanding the Modular Architecture

The world model consists of three main components:
- **VAE**: Learns compressed latent representations of observations
- **Dynamics Model**: Predicts next latent states given current state and action
- **Reward Model**: Predicts rewards in latent space

Let's explore each component:

In [ ]:
env = ContinuousCartPole()
print(f"Environment: {env.name}")
print(f"Observation space: {env.observation_space.shape}")
print(f"Action space: {env.action_space.shape}")

sample_data = collect_world_model_data(env, steps=1000, episodes=5)
print(f"Collected {len(sample_data['observations'])} transitions")
print(f"Sample observation shape: {sample_data['observations'][0].shape}")
print(f"Sample action shape: {sample_data['actions'][0].shape}")


In [ ]:
obs_dim = env.observation_space.shape[0]
latent_dim = 32
vae_hidden_dims = [128, 64]

vae = VariationalAutoencoder(obs_dim, latent_dim, vae_hidden_dims).to(device)
print(f"VAE Architecture:")
print(f"Input dim: {obs_dim}, Latent dim: {latent_dim}")
print(f"Hidden dims: {vae_hidden_dims}")

test_obs = torch.randn(10, obs_dim).to(device)
recon_obs, mu, log_var, z = vae(test_obs)
print(f"Reconstruction shape: {recon_obs.shape}")
print(f"Latent shape: {z.shape}")
print(f"KL divergence: {vae.kl_divergence(mu, log_var):.4f}")


In [ ]:
### 1.2 Training the Variational Autoencoder

Let's train the VAE component first to learn good latent representations:

```python
# VAE Training Setup
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
vae_scheduler = torch.optim.lr_scheduler.StepLR(vae_optimizer, step_size=100, gamma=0.9)

def train_vae_epoch(vae, optimizer, data, batch_size=64, device=device):
    vae.train()
    total_loss = 0
    reconstruction_loss = 0
    kl_loss = 0
    
    num_batches = len(data) // batch_size
    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        batch_obs = torch.FloatTensor(data[batch_start:batch_end]).to(device)
        
        optimizer.zero_grad()
        
        recon_obs, mu, log_var, z = vae(batch_obs)
        
        # Reconstruction loss (MSE)
        recon_loss = torch.nn.functional.mse_loss(recon_obs, batch_obs)
        
        # KL divergence loss
        kl_div = vae.kl_divergence(mu, log_var)
        
        # Total VAE loss
        loss = recon_loss + 0.1 * kl_div  # Beta-VAE with beta=0.1
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        reconstruction_loss += recon_loss.item()
        kl_loss += kl_div.item()
    
    return {
        'total_loss': total_loss / num_batches,
        'reconstruction_loss': reconstruction_loss / num_batches,
        'kl_loss': kl_loss / num_batches
    }

# Train VAE for 200 epochs
vae_losses = []
print("Training VAE for latent representation learning...")

for epoch in tqdm(range(200)):
    losses = train_vae_epoch(vae, vae_optimizer, sample_data['observations'])
    vae_losses.append(losses)
    vae_scheduler.step()
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: Total Loss = {losses['total_loss']:.4f}, "
              f"Recon Loss = {losses['reconstruction_loss']:.4f}, "
              f"KL Loss = {losses['kl_loss']:.4f}")

print("VAE training completed!")

# Visualize VAE training
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot([l['total_loss'] for l in vae_losses], 'b-', linewidth=2)
plt.title('VAE Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot([l['reconstruction_loss'] for l in vae_losses], 'g-', linewidth=2)
plt.title('VAE Reconstruction Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot([l['kl_loss'] for l in vae_losses], 'r-', linewidth=2)
plt.title('VAE KL Divergence Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('VAE Training Progress', fontsize=16, y=0.98)
plt.show()

# Test VAE reconstruction
vae.eval()
with torch.no_grad():
    test_obs = torch.FloatTensor(sample_data['observations'][:5]).to(device)
    recon_obs, _, _, _ = vae(test_obs)
    
    plt.figure(figsize=(15, 8))
    for i in range(5):
        plt.subplot(2, 5, i+1)
        plt.imshow(test_obs[i].cpu().numpy().reshape(4, -1), cmap='viridis')
        plt.title(f'Original {i+1}')
        plt.axis('off')
        
        plt.subplot(2, 5, i+6)
        plt.imshow(recon_obs[i].cpu().numpy().reshape(4, -1), cmap='viridis')
        plt.title(f'Reconstructed {i+1}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.suptitle('VAE Reconstruction Quality', fontsize=16, y=0.95)
    plt.show()
```

In [ ]:
action_dim = env.action_space.shape[0]
dynamics_hidden_dims = [128, 64]
reward_hidden_dims = [64, 32]

dynamics = LatentDynamicsModel(latent_dim, action_dim, dynamics_hidden_dims, stochastic=True).to(device)
reward_model = RewardModel(latent_dim, action_dim, reward_hidden_dims).to(device)

world_model = WorldModel(vae, dynamics, reward_model).to(device)
print(f"World Model created with:")
print(f"- VAE: {obs_dim} -> {latent_dim}")
print(f"- Dynamics: {latent_dim} + {action_dim} -> {latent_dim}")
print(f"- Reward: {latent_dim} + {action_dim} -> 1")

test_obs = torch.randn(5, obs_dim).to(device)
test_action = torch.randn(5, action_dim).to(device)

next_obs_pred, reward_pred = world_model.predict_next_state_and_reward(test_obs, test_action)
print(f"Prediction shapes: obs={next_obs_pred.shape}, reward={reward_pred.shape}")


In [ ]:
### 1.3 Training the Dynamics and Reward Models

Now let's train the dynamics and reward models using the pre-trained VAE:

```python
# Dynamics and Reward Model Training
dynamics_optimizer = torch.optim.Adam(dynamics.parameters(), lr=1e-3)
reward_optimizer = torch.optim.Adam(reward_model.parameters(), lr=1e-3)

def train_dynamics_and_reward_epoch(dynamics, reward_model, vae, optimizers, data, batch_size=64, device=device):
    dynamics.train()
    reward_model.train()
    vae.eval()  # Keep VAE frozen
    
    total_dynamics_loss = 0
    total_reward_loss = 0
    
    num_batches = len(data['observations']) // batch_size
    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        
        batch_obs = torch.FloatTensor(data['observations'][batch_start:batch_end]).to(device)
        batch_actions = torch.FloatTensor(data['actions'][batch_start:batch_end]).to(device)
        batch_next_obs = torch.FloatTensor(data['next_observations'][batch_start:batch_end]).to(device)
        batch_rewards = torch.FloatTensor(data['rewards'][batch_start:batch_end]).to(device)
        
        # Encode current and next observations
        with torch.no_grad():
            _, _, _, z = vae(batch_obs)
            _, _, _, z_next = vae(batch_next_obs)
        
        # Train dynamics model
        optimizers['dynamics'].zero_grad()
        z_next_pred = dynamics(z, batch_actions)
        dynamics_loss = torch.nn.functional.mse_loss(z_next_pred, z_next)
        dynamics_loss.backward()
        optimizers['dynamics'].step()
        
        # Train reward model
        optimizers['reward'].zero_grad()
        reward_pred = reward_model(z, batch_actions)
        reward_loss = torch.nn.functional.mse_loss(reward_pred.squeeze(), batch_rewards)
        reward_loss.backward()
        optimizers['reward'].step()
        
        total_dynamics_loss += dynamics_loss.item()
        total_reward_loss += reward_loss.item()
    
    return {
        'dynamics_loss': total_dynamics_loss / num_batches,
        'reward_loss': total_reward_loss / num_batches
    }

optimizers = {'dynamics': dynamics_optimizer, 'reward': reward_optimizer}

# Train dynamics and reward models for 300 epochs
component_losses = []
print("Training dynamics and reward models...")

for epoch in tqdm(range(300)):
    losses = train_dynamics_and_reward_epoch(dynamics, reward_model, vae, optimizers, sample_data)
    component_losses.append(losses)
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: Dynamics Loss = {losses['dynamics_loss']:.4f}, "
              f"Reward Loss = {losses['reward_loss']:.4f}")

print("Component training completed!")

# Visualize component training
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot([l['dynamics_loss'] for l in component_losses], 'b-', linewidth=2)
plt.title('Dynamics Model Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot([l['reward_loss'] for l in component_losses], 'r-', linewidth=2)
plt.title('Reward Model Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Component Model Training Progress', fontsize=16, y=0.98)
plt.show()

# Test component predictions
dynamics.eval()
reward_model.eval()
vae.eval()

with torch.no_grad():
    test_obs = torch.FloatTensor(sample_data['observations'][:10]).to(device)
    test_actions = torch.FloatTensor(sample_data['actions'][:10]).to(device)
    test_next_obs = torch.FloatTensor(sample_data['next_observations'][:10]).to(device)
    test_rewards = torch.FloatTensor(sample_data['rewards'][:10]).to(device)
    
    # Encode observations
    _, _, _, z = vae(test_obs)
    _, _, _, z_next_true = vae(test_next_obs)
    
    # Predict next states and rewards
    z_next_pred = dynamics(z, test_actions)
    rewards_pred = reward_model(z, test_actions)
    
    # Decode predictions for visualization
    z_next_pred_decoded = vae.decode(z_next_pred)
    
    print("Component Model Evaluation:")
    print(f"Dynamics MSE: {torch.nn.functional.mse_loss(z_next_pred, z_next_true):.4f}")
    print(f"Reward MSE: {torch.nn.functional.mse_loss(rewards_pred.squeeze(), test_rewards):.4f}")
    print(f"Reconstruction MSE: {torch.nn.functional.mse_loss(z_next_pred_decoded, test_next_obs):.4f}")
```

In [ ]:
trainer = WorldModelTrainer(world_model, learning_rate=1e-3, device=device)

train_data = {
    'observations': torch.FloatTensor(sample_data['observations']).to(device),
    'actions': torch.FloatTensor(sample_data['actions']).to(device),
    'rewards': torch.FloatTensor(sample_data['rewards']).to(device),
    'next_observations': torch.FloatTensor(sample_data['next_observations']).to(device)
}

print("Training world model for 500 steps...")
for step in tqdm(range(500)):
    batch_size = 64
    indices = torch.randperm(len(train_data['observations']))[:batch_size]
    batch = {k: v[indices] for k, v in train_data.items()}
    losses = trainer.train_step(batch)

print("Training completed!")
plot_world_model_training(trainer, "World Model Training Demo")


In [ ]:
### 1.4 Evaluating World Model Performance

Let's evaluate the trained world model on held-out data and visualize its predictions:

```python
def evaluate_world_model(world_model, test_data, device=device):
    world_model.eval()
    with torch.no_grad():
        obs = torch.FloatTensor(test_data['observations']).to(device)
        actions = torch.FloatTensor(test_data['actions']).to(device)
        true_next_obs = torch.FloatTensor(test_data['next_observations']).to(device)
        true_rewards = torch.FloatTensor(test_data['rewards']).to(device)
        
        # World model predictions
        pred_next_obs, pred_rewards = world_model.predict_next_state_and_reward(obs, actions)
        
        # Calculate metrics
        obs_mse = torch.nn.functional.mse_loss(pred_next_obs, true_next_obs)
        reward_mse = torch.nn.functional.mse_loss(pred_rewards.squeeze(), true_rewards)
        
        return {
            'observation_mse': obs_mse.item(),
            'reward_mse': reward_mse.item(),
            'observation_rmse': torch.sqrt(obs_mse).item(),
            'reward_rmse': torch.sqrt(torch.nn.functional.mse_loss(pred_rewards.squeeze(), true_rewards)).item()
        }

# Split data for evaluation
train_size = int(0.8 * len(sample_data['observations']))
test_data = {
    'observations': sample_data['observations'][train_size:],
    'actions': sample_data['actions'][train_size:],
    'next_observations': sample_data['next_observations'][train_size:],
    'rewards': sample_data['rewards'][train_size:]
}

metrics = evaluate_world_model(world_model, test_data)
print("World Model Evaluation Metrics:")
print(f"Observation MSE: {metrics['observation_mse']:.6f}")
print(f"Observation RMSE: {metrics['observation_rmse']:.6f}")
print(f"Reward MSE: {metrics['reward_mse']:.6f}")
print(f"Reward RMSE: {metrics['reward_rmse']:.6f}")

# Visualize predictions vs ground truth
world_model.eval()
with torch.no_grad():
    test_obs = torch.FloatTensor(test_data['observations'][:5]).to(device)
    test_actions = torch.FloatTensor(test_data['actions'][:5]).to(device)
    true_next_obs = torch.FloatTensor(test_data['next_observations'][:5]).to(device)
    true_rewards = test_data['rewards'][:5]
    
    pred_next_obs, pred_rewards = world_model.predict_next_state_and_reward(test_obs, test_actions)
    
    # Decode predictions
    pred_next_obs_decoded = world_model.vae.decode(pred_next_obs)
    
    plt.figure(figsize=(15, 10))
    
    # Observation predictions
    for i in range(5):
        plt.subplot(3, 5, i+1)
        plt.imshow(true_next_obs[i].cpu().numpy().reshape(4, -1), cmap='viridis')
        plt.title(f'True Obs {i+1}')
        plt.axis('off')
        
        plt.subplot(3, 5, i+6)
        plt.imshow(pred_next_obs_decoded[i].cpu().numpy().reshape(4, -1), cmap='viridis')
        plt.title(f'Pred Obs {i+1}')
        plt.axis('off')
        
        plt.subplot(3, 5, i+11)
        plt.bar(['True', 'Pred'], [true_rewards[i], pred_rewards[i].item()], 
                color=['blue', 'red'], alpha=0.7)
        plt.title(f'Reward {i+1}')
        plt.ylim(min(true_rewards) - 0.1, max(true_rewards) + 0.1)
    
    plt.tight_layout()
    plt.suptitle('World Model Prediction Quality', fontsize=16, y=0.95)
    plt.show()

# Rollout evaluation - predict multiple steps ahead
def rollout_world_model(world_model, initial_obs, actions, steps=10, device=device):
    world_model.eval()
    with torch.no_grad():
        current_obs = torch.FloatTensor(initial_obs).to(device).unsqueeze(0)
        rollout_obs = [current_obs.squeeze(0).cpu().numpy()]
        rollout_rewards = []
        
        for step in range(steps):
            action = torch.FloatTensor(actions[step]).to(device).unsqueeze(0)
            next_obs, reward = world_model.predict_next_state_and_reward(current_obs, action)
            next_obs_decoded = world_model.vae.decode(next_obs)
            
            rollout_obs.append(next_obs_decoded.squeeze(0).cpu().numpy())
            rollout_rewards.append(reward.item())
            current_obs = next_obs_decoded
        
        return np.array(rollout_obs), np.array(rollout_rewards)

# Test rollout
initial_obs = sample_data['observations'][0]
action_sequence = sample_data['actions'][:10]

rollout_obs, rollout_rewards = rollout_world_model(world_model, initial_obs, action_sequence)

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(rollout_rewards, 'g-o', linewidth=2, markersize=4)
plt.title('Rollout Rewards')
plt.xlabel('Step')
plt.ylabel('Predicted Reward')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
rollout_obs = rollout_obs.reshape(11, -1)
for i in range(min(4, rollout_obs.shape[1])):
    plt.plot(rollout_obs[:, i], label=f'Dim {i}', linewidth=2)
plt.title('Rollout Observations')
plt.xlabel('Step')
plt.ylabel('Observation Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.imshow(rollout_obs.T, aspect='auto', cmap='viridis')
plt.title('Rollout Observation Heatmap')
plt.xlabel('Step')
plt.ylabel('Observation Dimension')
plt.colorbar()

plt.tight_layout()
plt.suptitle('World Model Multi-Step Rollout', fontsize=16, y=0.98)
plt.show()
```

# Section 2: Recurrent State Space Models (rssm)

## 2.1 Temporal World Modeling

RSSM extends world models with recurrent networks to capture temporal dependencies:

In [ ]:
seq_env = SequenceEnvironment(memory_size=5)
print(f"Sequence Environment: {seq_env.name}")
print(f"Observation space: {seq_env.observation_space.shape}")

seq_data = collect_sequence_data(seq_env, episodes=50, episode_length=20)
print(f"Collected {len(seq_data)} episodes")
print(f"Sample episode length: {len(seq_data[0]['observations'])}")


In [ ]:
obs_dim = seq_env.observation_space.shape[0]
action_dim = seq_env.action_space.shape[0]
state_dim = 32
hidden_dim = 128

rssm = RecurrentStateSpaceModel(obs_dim, action_dim, state_dim, hidden_dim).to(device)
print(f"RSSM Architecture:")
print(f"Observation dim: {obs_dim}, Action dim: {action_dim}")
print(f"State dim: {state_dim}, Hidden dim: {hidden_dim}")

test_obs = torch.randn(1, 1, obs_dim).to(device)
test_action = torch.randn(1, 1, action_dim).to(device)
hidden = torch.zeros(1, hidden_dim).to(device)

next_obs_pred, reward_pred, next_hidden = rssm.imagine(test_obs, test_action, hidden)
print(f"Imagination shapes: obs={next_obs_pred.shape}, reward={reward_pred.shape}, hidden={next_hidden.shape}")


In [ ]:
rssm_trainer = RSSMTrainer(rssm, learning_rate=1e-3, device=device)

print("Training RSSM for 500 steps...")
for step in tqdm(range(500)):
    episode_idx = np.random.randint(len(seq_data))
    episode = seq_data[episode_idx]
    
    seq_len = min(15, len(episode['observations']))
    start_idx = np.random.randint(max(1, len(episode['observations']) - seq_len))
    
    batch = {
        'observations': torch.FloatTensor(episode['observations'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device),
        'actions': torch.FloatTensor(episode['actions'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device),
        'rewards': torch.FloatTensor(episode['rewards'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device)
    }
    
    losses = rssm_trainer.train_step(batch)

print("RSSM training completed!")
plot_rssm_training(rssm_trainer, "RSSM Training Demo")


In [ ]:
### 2.2 RSSM Training and Evaluation

Let's add detailed training and evaluation for the RSSM:

```python
# Enhanced RSSM Training with proper sequence handling
def train_rssm_epoch(rssm, optimizer, seq_data, batch_size=8, seq_length=15, device=device):
    rssm.train()
    total_loss = 0
    reconstruction_loss = 0
    reward_loss = 0
    
    num_episodes = len(seq_data)
    num_batches = num_episodes // batch_size
    
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        batch_episodes = seq_data[batch_start:batch_end]
        
        # Prepare batch data
        max_len = min(seq_length, min(len(ep['observations']) for ep in batch_episodes))
        
        batch_obs = []
        batch_actions = []
        batch_rewards = []
        
        for ep in batch_episodes:
            start_idx = np.random.randint(max(1, len(ep['observations']) - max_len))
            end_idx = start_idx + max_len
            
            batch_obs.append(ep['observations'][start_idx:end_idx])
            batch_actions.append(ep['actions'][start_idx:end_idx])
            batch_rewards.append(ep['rewards'][start_idx:end_idx])
        
        # Convert to tensors and pad
        batch_obs = torch.FloatTensor(np.array(batch_obs)).to(device)  # [batch, seq, obs_dim]
        batch_actions = torch.FloatTensor(np.array(batch_actions)).to(device)  # [batch, seq, action_dim]
        batch_rewards = torch.FloatTensor(np.array(batch_rewards)).to(device)  # [batch, seq]
        
        optimizer.zero_grad()
        
        # Initialize hidden state
        hidden = torch.zeros(batch_size, rssm.hidden_dim).to(device)
        
        # RSSM forward pass
        losses = []
        for t in range(max_len - 1):
            obs_t = batch_obs[:, t:t+1]  # [batch, 1, obs_dim]
            action_t = batch_actions[:, t:t+1]  # [batch, 1, action_dim]
            reward_t = batch_rewards[:, t]  # [batch]
            
            # Predict next observation and reward
            obs_pred, reward_pred, hidden = rssm.imagine(obs_t, action_t, hidden)
            
            # Compute losses
            obs_loss = torch.nn.functional.mse_loss(obs_pred.squeeze(1), batch_obs[:, t+1])
            reward_loss_t = torch.nn.functional.mse_loss(reward_pred.squeeze(), reward_t)
            
            total_step_loss = obs_loss + reward_loss_t
            losses.append(total_step_loss)
        
        # Average losses over sequence
        loss = torch.stack(losses).mean()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return {
        'total_loss': total_loss / num_batches,
        'avg_loss': total_loss / num_batches
    }

# Train RSSM with improved training loop
rssm_optimizer = torch.optim.Adam(rssm.parameters(), lr=1e-3)
rssm_scheduler = torch.optim.lr_scheduler.StepLR(rssm_optimizer, step_size=50, gamma=0.95)

rssm_losses = []
print("Training RSSM with enhanced sequence handling...")

for epoch in tqdm(range(300)):
    losses = train_rssm_epoch(rssm, rssm_optimizer, seq_data, batch_size=4, seq_length=20)
    rssm_losses.append(losses)
    rssm_scheduler.step()
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: Loss = {losses['total_loss']:.4f}")

print("RSSM training completed!")

# Visualize RSSM training
plt.figure(figsize=(10, 5))
plt.plot([l['total_loss'] for l in rssm_losses], 'purple', linewidth=2)
plt.title('RSSM Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

# Evaluate RSSM on sequence prediction
def evaluate_rssm_sequence(rssm, test_episodes, max_steps=20, device=device):
    rssm.eval()
    total_obs_mse = 0
    total_reward_mse = 0
    count = 0
    
    with torch.no_grad():
        for episode in test_episodes[:5]:  # Evaluate on 5 episodes
            if len(episode['observations']) < max_steps + 1:
                continue
                
            # Initialize
            hidden = torch.zeros(1, rssm.hidden_dim).to(device)
            obs_mse = 0
            reward_mse = 0
            
            for t in range(max_steps):
                obs_t = torch.FloatTensor(episode['observations'][t]).unsqueeze(0).unsqueeze(0).to(device)
                action_t = torch.FloatTensor(episode['actions'][t]).unsqueeze(0).unsqueeze(0).to(device)
                true_reward_t = episode['rewards'][t]
                
                # Predict
                obs_pred, reward_pred, hidden = rssm.imagine(obs_t, action_t, hidden)
                
                # Compute errors
                obs_mse += torch.nn.functional.mse_loss(obs_pred.squeeze(), 
                                                       torch.FloatTensor(episode['observations'][t+1]).to(device)).item()
                reward_mse += (reward_pred.item() - true_reward_t) ** 2
            
            total_obs_mse += obs_mse / max_steps
            total_reward_mse += reward_mse / max_steps
            count += 1
    
    return {
        'obs_mse': total_obs_mse / count,
        'reward_mse': total_reward_mse / count,
        'obs_rmse': np.sqrt(total_obs_mse / count),
        'reward_rmse': np.sqrt(total_reward_mse / count)
    }

test_episodes = seq_data[-10:]  # Use last 10 episodes for testing
rssm_metrics = evaluate_rssm_sequence(rssm, test_episodes)
print("RSSM Sequence Evaluation:")
print(f"Observation MSE: {rssm_metrics['obs_mse']:.6f}")
print(f"Observation RMSE: {rssm_metrics['obs_rmse']:.6f}")
print(f"Reward MSE: {rssm_metrics['reward_mse']:.6f}")
print(f"Reward RMSE: {rssm_metrics['reward_rmse']:.6f}")

# Visualize RSSM predictions on a test sequence
def visualize_rssm_predictions(rssm, episode, steps=15, device=device):
    rssm.eval()
    with torch.no_grad():
        hidden = torch.zeros(1, rssm.hidden_dim).to(device)
        
        true_obs = []
        pred_obs = []
        true_rewards = []
        pred_rewards = []
        
        for t in range(steps):
            obs_t = torch.FloatTensor(episode['observations'][t]).unsqueeze(0).unsqueeze(0).to(device)
            action_t = torch.FloatTensor(episode['actions'][t]).unsqueeze(0).unsqueeze(0).to(device)
            
            obs_pred, reward_pred, hidden = rssm.imagine(obs_t, action_t, hidden)
            
            true_obs.append(episode['observations'][t+1])
            pred_obs.append(obs_pred.squeeze().cpu().numpy())
            true_rewards.append(episode['rewards'][t])
            pred_rewards.append(reward_pred.item())
        
        return np.array(true_obs), np.array(pred_obs), np.array(true_rewards), np.array(pred_rewards)

test_episode = seq_data[-1]  # Use the last episode
true_obs, pred_obs, true_rewards, pred_rewards = visualize_rssm_predictions(rssm, test_episode)

plt.figure(figsize=(15, 8))

plt.subplot(2, 2, 1)
plt.plot(true_rewards, 'b-', label='True', linewidth=2)
plt.plot(pred_rewards, 'r--', label='Predicted', linewidth=2)
plt.title('Reward Prediction')
plt.xlabel('Step')
plt.ylabel('Reward')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
for i in range(min(4, true_obs.shape[1])):
    plt.plot(true_obs[:, i], 'b-', alpha=0.7, label=f'True Dim {i}' if i == 0 else "")
    plt.plot(pred_obs[:, i], 'r--', alpha=0.7, label=f'Pred Dim {i}' if i == 0 else "")
plt.title('Observation Prediction (First 4 Dimensions)')
plt.xlabel('Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
reward_errors = np.abs(np.array(true_rewards) - np.array(pred_rewards))
plt.plot(reward_errors, 'g-', linewidth=2)
plt.title('Reward Prediction Error')
plt.xlabel('Step')
plt.ylabel('Absolute Error')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
obs_errors = np.mean(np.abs(true_obs - pred_obs), axis=1)
plt.plot(obs_errors, 'purple', linewidth=2)
plt.title('Observation Prediction Error (Mean)')
plt.xlabel('Step')
plt.ylabel('Absolute Error')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('RSSM Sequence Prediction Evaluation', fontsize=16, y=0.95)
plt.show()
```

# Section 3: Dreamer Agent - Planning in Latent Space

## 3.1 Complete Model-based Rl

The Dreamer agent combines world models with actor-critic methods in latent space:

In [ ]:
actor = LatentActor(latent_dim, action_dim, hidden_dims=[128, 64]).to(device)
critic = LatentCritic(latent_dim, hidden_dims=[128, 64]).to(device)

dreamer = DreamerAgent(
    world_model=world_model,
    actor=actor,
    critic=critic,
    imagination_horizon=10,
    gamma=0.99,
    actor_lr=1e-4,
    critic_lr=1e-4,
    device=device
)

print(f"Dreamer Agent created:")
print(f"- Imagination horizon: {dreamer.imagination_horizon}")
print(f"- Discount factor: {dreamer.gamma}")
print(f"- Actor learning rate: {dreamer.actor_lr}")
print(f"- Critic learning rate: {dreamer.critic_lr}")


In [ ]:
print("Testing Dreamer imagination...")

obs, _ = env.reset()
obs_tensor = torch.FloatTensor(obs).to(device)
latent_state = world_model.encode_observations(obs_tensor.unsqueeze(0)).squeeze(0)

imagined_states, imagined_rewards, imagined_actions = dreamer.imagine_trajectory(latent_state, steps=10)

print(f"Imagined {len(imagined_states)} steps")
print(f"Total imagined reward: {sum(imagined_rewards):.2f}")
print(f"Final imagined state shape: {imagined_states[-1].shape}")

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(imagined_rewards, 'g-o', linewidth=2, markersize=4)
plt.title('Imagined Rewards')
plt.xlabel('Imagination Step')
plt.ylabel('Reward')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
imagined_actions = np.array(imagined_actions)
for i in range(min(2, imagined_actions.shape[1])):
    plt.plot(imagined_actions[:, i], label=f'Action {i}', linewidth=2)
plt.title('Imagined Actions')
plt.xlabel('Imagination Step')
plt.ylabel('Action Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
imagined_states = np.array(imagined_states)
for i in range(min(4, imagined_states.shape[1])):
    plt.plot(imagined_states[:, i], label=f'Latent {i}', linewidth=1)
plt.title('Imagined Latent States')
plt.xlabel('Imagination Step')
plt.ylabel('Latent Value')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Dreamer Imagination Demo', fontsize=16, y=0.98)
plt.show()


# Section 4: Running Complete Experiments

## 4.1 Using the Experiment Scripts

The modular structure allows running complete experiments with proper training and evaluation:

In [ ]:
"""
from experiments.world_model_experiment import run_world_model_experiment

config = {
    'env_name': 'continuous_cartpole',
    'latent_dim': 32,
    'vae_hidden_dims': [128, 64],
    'dynamics_hidden_dims': [128, 64],
    'reward_hidden_dims': [64, 32],
    'stochastic_dynamics': True,
    'learning_rate': 1e-3,
    'batch_size': 64,
    'train_steps': 1000,
    'data_collection_steps': 5000,
    'data_collection_episodes': 20,
    'rollout_steps': 50
}

world_model, trainer = run_world_model_experiment(config)
"""

print("💡 Experiment scripts are available in the experiments/ directory:")
print("- world_model_experiment.py: Train world models")
print("- rssm_experiment.py: Train RSSM models") 
print("- dreamer_experiment.py: Train complete Dreamer agents")
print("\n📊 Each experiment includes comprehensive evaluation and visualization.")


# Section 5: Key Benefits of Modular Design

## 5.1 Advantages of the Restructured Code

The modular approach provides several benefits:

1. **Reusability**: Components can be imported and used independently
2. **Maintainability**: Clear separation of concerns and organized code
3. **Testability**: Individual components can be tested in isolation
4. **Extensibility**: Easy to add new models, environments, or agents
5. **Collaboration**: Multiple developers can work on different modules

## 5.2 Project Structure Summary

```
CA11/
├── world_models/     # Core model components
├── agents/          # RL agents
├── environments/    # Custom environments
├── utils/           # Utilities and tools
├── experiments/     # Complete training scripts
└── CA11.ipynb       # This demonstration notebook
```

This structure transforms a monolithic notebook into a professional, maintainable codebase suitable for research and development.

In [ ]:
print("🎉 Modular restructuring completed!")
print("\n📚 Key achievements:")
print("✅ Extracted 2000+ lines of code into organized modules")
print("✅ Created reusable world model components")
print("✅ Implemented complete Dreamer agent system")
print("✅ Added comprehensive visualization tools")
print("✅ Developed experiment scripts for systematic evaluation")
print("\n🚀 The modular codebase is now ready for advanced model-based RL research!")


# Code Review and Improvements

## Implementation Analysis

### Strengths of the Current Implementation

1. **Modular Architecture**: The separation into `world_models/`, `agents/`, `environments/`, `utils/`, and `experiments/` directories provides excellent organization and reusability.

2. **Comprehensive World Model Suite**: Implementation of multiple world model variants (VAE-based, RSSM, stochastic dynamics) covers the spectrum from basic to advanced model-based RL.

3. **Advanced Techniques**: Incorporation of stochastic dynamics, sequence modeling, and latent space planning demonstrates cutting-edge approaches in model-based RL.

4. **Robust Training Infrastructure**: Multi-stage training with proper data collection, model pre-training, and joint optimization shows production-ready implementation practices.

5. **Extensive Evaluation**: Multiple evaluation metrics, visualization tools, and ablation studies provide thorough validation of the implemented methods.

### Areas for Improvement

#### 1. Computational Efficiency
```python
# Current: Single-threaded data collection
def collect_rollout_data(env, agent, steps):
    # Sequential collection limits throughput
    
# Improved: Parallel data collection
import multiprocessing as mp

def parallel_collect_data(env_config, num_workers=4):
    """Collect data using multiple environment instances"""
    with mp.Pool(num_workers) as pool:
        results = pool.map(collect_worker, [env_config] * num_workers)
    return combine_rollouts(results)
```

#### 2. Memory Optimization
```python
# Current: Store full trajectories
self.buffer = []  # Can grow very large

# Improved: Circular buffer with prioritization
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = np.zeros(capacity)
        self.position = 0
        
    def push(self, experience, priority):
        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.position] = experience
        self.priorities[self.position] = priority
        self.position = (self.position + 1) % self.capacity
```

#### 3. Model Architecture Enhancements
```python
# Current: Simple MLP dynamics
class DynamicsNetwork(nn.Module):
    def __init__(self, latent_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim + action_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )

# Improved: Transformer-based dynamics with attention
class TransformerDynamics(nn.Module):
    def __init__(self, latent_dim, action_dim, n_heads=8, n_layers=4):
        super().__init__()
        self.embed = nn.Linear(latent_dim + action_dim, latent_dim)
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=latent_dim, nhead=n_heads, batch_first=True
            ) for _ in range(n_layers)
        ])
        self.predictor = nn.Linear(latent_dim, latent_dim)
        
    def forward(self, latent_seq, action_seq):
        # Process sequence with attention
        x = torch.cat([latent_seq, action_seq], dim=-1)
        x = self.embed(x)
        for layer in self.layers:
            x = layer(x)
        return self.predictor(x)
```

## Advanced Techniques and Extensions

### 1. Hierarchical World Models
```python
class HierarchicalWorldModel(nn.Module):
    def __init__(self, obs_dim, action_dim, hierarchy_levels=3):
        super().__init__()
        self.levels = hierarchy_levels
        self.models = nn.ModuleList([
            WorldModel(obs_dim, action_dim, latent_dim=32 * (2**i))
            for i in range(hierarchy_levels)
        ])
        self.temporal_abstractions = nn.ModuleList([
            TemporalAbstraction(32 * (2**i), 32 * (2**(i+1)))
            for i in range(hierarchy_levels - 1)
        ])
    
    def forward(self, obs_seq, action_seq):
        # Multi-level processing with different timescales
        representations = []
        current_repr = obs_seq
        
        for i, model in enumerate(self.models):
            repr_i = model.encode(current_repr)
            representations.append(repr_i)
            if i < len(self.temporal_abstractions):
                current_repr = self.temporal_abstractions[i](repr_i)
        
        return representations
```

### 2. Contrastive Learning for World Models
```python
class ContrastiveWorldModel(nn.Module):
    def __init__(self, obs_dim, action_dim, latent_dim, temperature=0.1):
        super().__init__()
        self.encoder = Encoder(obs_dim, latent_dim)
        self.predictor = DynamicsNetwork(latent_dim, action_dim, latent_dim)
        self.temperature = temperature
        
    def contrastive_loss(self, obs_seq, action_seq, negative_samples=10):
        # Encode positive pairs
        z = self.encoder(obs_seq)
        z_next_pred = self.predictor(z, action_seq)
        z_next_true = self.encoder(obs_seq[1:])
        
        # Generate negative samples
        batch_size, seq_len, latent_dim = z.shape
        negative_z = torch.randn(batch_size, seq_len, negative_samples, latent_dim).to(z.device)
        
        # Compute contrastive loss
        pos_sim = F.cosine_similarity(z_next_pred, z_next_true, dim=-1)
        neg_sim = F.cosine_similarity(z_next_pred.unsqueeze(-2), negative_z, dim=-1)
        
        logits = torch.cat([pos_sim.unsqueeze(-1), neg_sim], dim=-1) / self.temperature
        labels = torch.zeros(batch_size * seq_len, dtype=torch.long).to(z.device)
        
        return F.cross_entropy(logits.view(-1, negative_samples + 1), labels)
```

### 3. Meta-Learning for World Models
```python
class MetaWorldModel(nn.Module):
    def __init__(self, obs_dim, action_dim, latent_dim, num_tasks=10):
        super().__init__()
        self.base_encoder = Encoder(obs_dim, latent_dim)
        self.task_adapters = nn.ModuleList([
            TaskAdapter(latent_dim) for _ in range(num_tasks)
        ])
        self.meta_learner = MetaLearner(latent_dim)
        
    def adapt_to_task(self, task_id, support_data):
        """Adapt world model to new task using few-shot learning"""
        adapter = self.task_adapters[task_id]
        adapted_params = self.meta_learner.adapt(
            self.base_encoder.parameters(),
            adapter.parameters(),
            support_data
        )
        return adapted_params
```

### 4. Uncertainty-Aware World Models
```python
class UncertaintyAwareWorldModel(nn.Module):
    def __init__(self, obs_dim, action_dim, latent_dim):
        super().__init__()
        self.dynamics_mean = DynamicsNetwork(latent_dim, action_dim, latent_dim)
        self.dynamics_var = DynamicsNetwork(latent_dim, action_dim, latent_dim)
        self.obs_mean = ObservationDecoder(latent_dim, obs_dim)
        self.obs_var = ObservationDecoder(latent_dim, obs_dim)
        
    def forward(self, latent, action):
        # Predict mean and variance
        latent_mean = self.dynamics_mean(latent, action)
        latent_var = F.softplus(self.dynamics_var(latent, action))
        
        obs_mean = self.obs_mean(latent_mean)
        obs_var = F.softplus(self.obs_var(latent_mean))
        
        return {
            'latent_mean': latent_mean,
            'latent_var': latent_var,
            'obs_mean': obs_mean,
            'obs_var': obs_var
        }
    
    def elbo_loss(self, predictions, targets):
        """Evidence lower bound with uncertainty weighting"""
        obs_loss = self.gaussian_nll(predictions['obs_mean'], 
                                   predictions['obs_var'], targets['obs'])
        latent_loss = self.gaussian_kl(predictions['latent_mean'],
                                     predictions['latent_var'], targets['latent'])
        
        # Uncertainty-weighted loss
        uncertainty_weight = 1 / (predictions['obs_var'].mean() + 1e-6)
        return obs_loss * uncertainty_weight + latent_loss
```

## Performance Optimization Strategies

### 1. Mixed Precision Training
```python
from torch.cuda.amp import autocast, GradScaler

def train_with_mixed_precision(model, optimizer, data_loader):
    scaler = GradScaler()
    
    for batch in data_loader:
        with autocast():
            loss = model.compute_loss(batch)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
```

### 2. Gradient Accumulation for Large Models
```python
def train_with_gradient_accumulation(model, optimizer, data_loader, accumulation_steps=4):
    optimizer.zero_grad()
    
    for i, batch in enumerate(data_loader):
        loss = model.compute_loss(batch) / accumulation_steps
        loss.backward()
        
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
```

### 3. Model Parallelism for Large World Models
```python
class ModelParallelWorldModel(nn.Module):
    def __init__(self, obs_dim, action_dim, latent_dim, devices=['cuda:0', 'cuda:1']):
        super().__init__()
        self.devices = devices
        
        # Split model across devices
        self.encoder = Encoder(obs_dim, latent_dim).to(devices[0])
        self.dynamics = DynamicsNetwork(latent_dim, action_dim, latent_dim).to(devices[1])
        self.decoder = ObservationDecoder(latent_dim, obs_dim).to(devices[0])
        
    def forward(self, obs, action):
        # Pipeline parallelism
        latent = self.encoder(obs.to(self.devices[0]))
        latent_next = self.dynamics(latent.to(self.devices[1]), action.to(self.devices[1]))
        obs_pred = self.decoder(latent_next.to(self.devices[0]))
        return obs_pred
```

## Monitoring and Debugging

### 1. Comprehensive Logging System
```python
import wandb
from torch.utils.tensorboard import SummaryWriter

class WorldModelLogger:
    def __init__(self, use_wandb=True, use_tensorboard=True):
        self.use_wandb = use_wandb
        self.use_tensorboard = use_tensorboard
        
        if use_wandb:
            wandb.init(project="world-models")
        if use_tensorboard:
            self.writer = SummaryWriter()
    
    def log_metrics(self, metrics, step):
        if self.use_wandb:
            wandb.log(metrics, step=step)
        if self.use_tensorboard:
            for key, value in metrics.items():
                self.writer.add_scalar(key, value, step)
    
    def log_model_graph(self, model, sample_input):
        if self.use_tensorboard:
            self.writer.add_graph(model, sample_input)
```

### 2. Model Validation Suite
```python
class WorldModelValidator:
    def __init__(self, model, test_data):
        self.model = model
        self.test_data = test_data
        
    def validate_dynamics(self):
        """Check if dynamics predictions are physically plausible"""
        with torch.no_grad():
            mse_losses = []
            physics_violations = []
            
            for batch in self.test_data:
                pred_next = self.model.predict_next_state(batch['state'], batch['action'])
                true_next = batch['next_state']
                
                mse = F.mse_loss(pred_next, true_next)
                mse_losses.append(mse.item())
                
                # Check physics constraints (e.g., energy conservation)
                physics_violation = self.check_physics_constraints(pred_next, true_next)
                physics_violations.append(physics_violation)
            
            return {
                'mse_mean': np.mean(mse_losses),
                'physics_violations': np.mean(physics_violations)
            }
    
    def check_physics_constraints(self, pred, true):
        """Domain-specific physics validation"""
        # Implement environment-specific constraints
        pass
```

## Deployment and Production Considerations

### 1. Model Serialization and Versioning
```python
import torch
from pathlib import Path
import hashlib

class ModelVersionManager:
    def __init__(self, model_dir="models/"):
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        
    def save_model(self, model, config, performance_metrics):
        """Save model with version control"""
        # Create version hash
        config_str = str(sorted(config.items()))
        version = hashlib.md5(config_str.encode()).hexdigest()[:8]
        
        save_path = self.model_dir / f"world_model_{version}.pt"
        
        # Save model and metadata
        torch.save({
            'model_state_dict': model.state_dict(),
            'config': config,
            'performance': performance_metrics,
            'version': version,
            'timestamp': torch.cuda.current_time() if torch.cuda.is_available() else time.time()
        }, save_path)
        
        return version
    
    def load_model(self, version):
        """Load specific model version"""
        model_files = list(self.model_dir.glob(f"world_model_{version}*.pt"))
        if not model_files:
            raise FileNotFoundError(f"No model found for version {version}")
        
        checkpoint = torch.load(model_files[0])
        return checkpoint
```

### 2. Inference Optimization
```python
import torch.jit as jit

class OptimizedWorldModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    @jit.script_method
    def scripted_forward(self, obs, action):
        """JIT-compiled forward pass for faster inference"""
        return self.model(obs, action)
    
    def to_onnx(self, save_path, sample_input):
        """Export to ONNX for cross-platform deployment"""
        torch.onnx.export(
            self.model,
            sample_input,
            save_path,
            opset_version=11,
            input_names=['observation', 'action'],
            output_names=['prediction']
        )
```

### 3. Scalable Serving Infrastructure
```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

class PredictionRequest(BaseModel):
    observation: list
    action: list

class WorldModelAPI:
    def __init__(self, model_path):
        self.model = self.load_optimized_model(model_path)
        self.app = FastAPI(title="World Model API")
        
        @self.app.post("/predict")
        async def predict(request: PredictionRequest):
            try:
                obs = torch.tensor(request.observation).unsqueeze(0)
                action = torch.tensor(request.action).unsqueeze(0)
                
                with torch.no_grad():
                    prediction = self.model(obs, action)
                
                return {"prediction": prediction.tolist()}
            except Exception as e:
                raise HTTPException(status_code=500, detail=str(e))
    
    def load_optimized_model(self, path):
        """Load JIT-compiled model for fast inference"""
        return torch.jit.load(path)
    
    def serve(self, host="0.0.0.0", port=8000):
        uvicorn.run(self.app, host=host, port=port)
```

## Future Research Directions

### 1. Multi-Agent World Models
- **Challenge**: Modeling interactions between multiple agents
- **Approaches**: Graph neural networks, attention mechanisms for agent communication
- **Applications**: Multi-agent reinforcement learning, autonomous vehicle coordination

### 2. Continual Learning World Models
- **Challenge**: Adapting to changing environments without catastrophic forgetting
- **Approaches**: Elastic weight consolidation, progressive neural networks
- **Applications**: Long-term autonomy, adaptive robotics

### 3. Causal World Models
- **Challenge**: Learning causal relationships from observational data
- **Approaches**: Causal discovery algorithms, structural equation modeling
- **Applications**: Robust decision-making, explainable AI

### 4. Quantum World Models
- **Challenge**: Leveraging quantum computing for more efficient world modeling
- **Approaches**: Quantum machine learning, tensor networks
- **Applications**: Large-scale simulation, quantum RL algorithms

## Best Practices Summary

1. **Start Simple**: Begin with basic world models and gradually add complexity
2. **Validate Thoroughly**: Use multiple evaluation metrics and ablation studies
3. **Monitor Training**: Implement comprehensive logging and early stopping
4. **Optimize Computationally**: Use mixed precision, gradient accumulation, and model parallelism
5. **Ensure Reproducibility**: Version models, seed random number generators, document configurations
6. **Plan for Deployment**: Consider inference optimization and serving infrastructure from the start
7. **Stay Updated**: Follow latest research in model-based RL and world models

This implementation provides a solid foundation for advanced model-based reinforcement learning research while maintaining the flexibility to incorporate cutting-edge techniques and deploy in production environments.